In [1]:
import os
from matplotlib import pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
import shutil
from experiment.lab2._config_ import LabSpace

sumMeasureDir = os.path.join(LabSpace, '.measure')

## Read network

In [3]:
from experiment.functions import func_get_lab_graph
from loguru import logger
from experiment.lab2._config_ import SoftwareConfit, LabSpace

GraphDict = dict()

for softwareName, (status, language, src_dir) in SoftwareConfit.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')
        graphDir = os.path.join(labDir, 'graph')

        if not os.path.isdir(labDir):
            os.makedirs(labDir, exist_ok=True)

        logger.info(f'>>> {softwareName} >>> read network objects')
        labGraph, labelDict, _ = func_get_lab_graph(modelDir, labDir, graphDir, softwareName, True)
        GraphDict[softwareName] = labGraph

In [4]:
print(GraphDict.keys())

## Degree analysis of software

In [14]:
from experiment.algorithms.distribution.powerlaw_ana import powerlaw_fit
import powerlaw
import numpy as np

degreeInfos = dict()
for softwareName, G in GraphDict.items():
    data = np.asarray(list(dict(G.degree()).values()))
    fit, info, X, Y, logX, logY = powerlaw_fit(data, plot=True, kMinPlot=False, title=f'Degree distribution of {softwareName}')
    degreeInfos[softwareName] = info

for softwareName, degreeInfo in degreeInfos.items():
    if degreeInfo['D'] < 0.05:
        print(f'{softwareName} ------- {degreeInfo}')
    else:
        print(f'{softwareName} +++++++ {degreeInfo}')

K_min 分析

In [6]:
D_list = []
for x in np.linspace(1,51,26):
    fit = powerlaw.Fit(data, xmin = x)
    D_list.append(fit.power_law.D)

plt.plot(np.linspace(1,51,26), D_list, "r-")
plt.xlabel("$K_{min}$")
plt.ylabel("$D$")
plt.yscale("log")